In [1]:
import json
import os
import pickle
import time
import math

import numpy as np
import pandas as pd
from PIL import Image
from scipy.spatial.distance import mahalanobis
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from sklearn.manifold import TSNE

import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from my_utils import *

# === Local application-specific imports ===
from utils_uf_methods import *


# Loading Dataset for Training and Hold-Out

Below I load into memory the data that will be used for training and later to compute statistics useful for OOD detection.  
These data are observations collected from a reinforcement learning agent running on the standard environment after training is completed.  
They consist of sequences divided into episodes, which are then shuffled together.

The data will also be split into two groups based on the outcome of the recorded episode.  
My intention is to train two different models: one trained on all the data, and a second trained only on states from episodes where the agent successfully reached the goal.

This distinction is important because the agent does not always succeed, and when it fails, episodes tend to last many more steps, during which the agent either gets stuck in walls or cannot escape from a local minimum.  
As a result, the dataset contains many "unusual" or outlier-like states, and it will be interesting to study how the absence of these states affects the performance of the model and the overall analysis.


In [ ]:
# Select GPU if available, otherwise fallback to CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

if torch.cuda.is_available():
    # Print CUDA-related details for reproducibility/debugging
    print(f"PyTorch CUDA version: {torch.version.cuda}")
    print(f"cuDNN enabled: {torch.backends.cudnn.enabled}")
    print(f"GPU name: {torch.cuda.get_device_name(0)}")
else:
    # Inform user when running only on CPU
    print("CUDA not available. Running on CPU.")

In [ ]:
# Load dataset from JSON file
with open('./u_e_test/datasets/standard_2200087.json') as f:
    raw_dataset = json.load(f)

# raw_dataset is a list of episodes
# Each episode is itself a list of steps
# Each step has the format:
# [observation_list, success_flag, episode_length, ...]

In [7]:
states = torch.tensor([step[:-2] for episodio in raw_dataset for step in episodio]) # remove action

In [8]:
# proportion: tr=0.6 / val=0.2 / ts=0.2 
states = split_dataset(states)

In [ ]:
# Small constant to prevent division by zero in std
STD_EPSILON = 1e-6

# Compute mean and std from TRAIN set only (index 0)
states_mean = states[0].mean(dim=0, keepdim=True)
states_std = states[0].std(dim=0, keepdim=True) + STD_EPSILON

# Keep a copy of calibration set without standardization
calibration_not_std = states[2]

# Standardize all splits (train, val, cal, test)
for i in range(len(states)):
    states[i] = (states[i] - states_mean) / states_std

# Wrap splits into dataset objects
states_dataset = (
    FlatDataset(states[0]),  # train
    FlatDataset(states[1]),  # val
    FlatDataset(states[2]),  # cal
    FlatDataset(states[3])   # test
)
calibration_not_std_dataset = FlatDataset(calibration_not_std)

# Create dataloaders
states_loader = (
    DataLoader(states_dataset[0], batch_size=256, shuffle=True),
    DataLoader(states_dataset[1], batch_size=256, shuffle=True),
    DataLoader(states_dataset[2], batch_size=256, shuffle=True),
    DataLoader(states_dataset[3], batch_size=256, shuffle=True)
)
calibration_not_std_loader = DataLoader(calibration_not_std_dataset, batch_size=256, shuffle=True)


# Training

In [ ]:
# Hyperparameter grids
lrs = [0.0005, 0.0001, 0.001]   # learning rates
wds = [1e-6, 1e-5, 1e-7]        # weight decays

'''
# Grid search over learning rates and weight decays
for l in lrs:
    for w in wds:
        train_rnd(
            "rnd",              # prefix name
            states_loader[0],   # train set
            states_loader[1],   # validation set
            device,             # CPU/GPU
            [128,128,128],      # hidden layers
            500,                # max epochs
            l,                  # learning rate
            w,                  # weight decay
            8                   # patience
        )
'''


# test

In [ ]:
# Load saved RND model parameters
rnd_params = torch.load('./u_e_test/ood_models/rnd_7346509_uptight_vortex.pth')

# Initialize networks: predictor (trainable) and source (fixed random target)
rnd_source = RNDNetwork(96, [128,128,128])
rnd_predictor = RNDNetwork(96, [128,128,128])

# Load trained weights into the predictor and fixed weights into the source
rnd_predictor.load_state_dict(rnd_params['predictor_state_dict'])
rnd_source.load_state_dict(rnd_params['random_state_dict'])

# Set both networks to evaluation mode (disable dropout, BN updates, etc.)
rnd_predictor.eval()
rnd_source.eval()


In [ ]:
def compute_uncertainties_rnd(source, predictor, data_loader):
    """
    Compute RND uncertainties as the squared difference between
    the fixed random network (source) and the trained predictor.
    """
    all_diffs = []
    source.eval()
    predictor.eval()
    
    with torch.no_grad():
        for x in data_loader:
            # Forward pass through both networks
            pred_source = source(x)
            pred_predictor = predictor(x)
            
            # Element-wise squared difference
            diff = (pred_source - pred_predictor) ** 2
            
            # Aggregate across features → one uncertainty score per sample
            diff = diff.sum(dim=1) * 100  # scaling factor
            
            all_diffs.append(diff.detach().cpu())
    
    # Concatenate all batches into a single array
    all_diffs = torch.cat(all_diffs, dim=0)
    return all_diffs.numpy()


In [ ]:
def print_percentiles(name, data):
    """
    Compute and print selected percentiles of the input data.
    """
    # Percentiles to compute
    percentiles = torch.tensor(
        [1,10,20,30,40,50,60,65,70,75,80,85,90,95,99],
        dtype=torch.float32
    )

    # Ensure input is a torch tensor
    if not isinstance(data, torch.Tensor):
        data = torch.from_numpy(data)

    # Compute percentile values
    values = torch.quantile(data, percentiles / 100.0)

    # Pair percentile values with their results
    results = list(zip(percentiles.tolist(), values.tolist()))
    
    # Print formatted output
    print(f"{name} percentiles:")
    for x in results:
        print(f"  {x[0]}th percentile: {x[1]}")
        
    return results


In [ ]:
prct = print_percentiles('rnd',  compute_uncertainties_rnd(rnd_source, rnd_predictor, states_loader[2]))

In [ ]:
# RND model configuration
args = {
    'input_dim': 96,
    'hidden_layers': [128, 128, 128],
}

# Save RND checkpoint with source/predictor weights, normalization stats, and percentiles
torch.save({
    'model_parameters': [rnd_source.state_dict(),  # fixed random network
                         rnd_predictor.state_dict()],  # trained predictor
    'model_args': args,                              # architecture settings
    'input_mean': states_mean,                       # normalization mean (train set)
    'input_std': states_std,                         # normalization std (train set)
    'percentiles': prct                              # uncertainty distribution
}, './u_e_test/rnd_method.pth')
